<a href="https://www.kaggle.com/code/abd0kamel/2023-10-sprint2-kaggle?scriptVersionId=148970804" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# WLASL Dataset Sign Language Model

In [1]:
import os
import json
import numpy as np

In [2]:
# Load the JSON file
with open('/kaggle/input/wlasl2000-resized/wlasl-complete/WLASL_v0.3.json', 'r') as json_file:
    data = json.load(json_file)

# Words to filter by
words_to_filter = ["where","hello","thank you", "go","stop","here","traffic","good","bad","today"]

# Filter the data
filtered_data = [item for item in data if item.get("gloss") in words_to_filter]

# Extract the first 5 "bbox" elements from each item
for item in filtered_data:
    item["instances"] = item["instances"][:5]

# Save the filtered data with the first 5 "bbox" elements to a JSON file
with open('filtered_data_with_5_bbox.json', 'w') as output_file:
    json.dump(filtered_data, output_file, indent=4)

In [3]:
with open('/kaggle/working/filtered_data_with_5_bbox.json', 'r') as f:
    data = json.load(f)

In [4]:
len(data[0]['instances'])

5

In [5]:
gloss_list = [] # done
video_name_list = [] # done
start_frame = [] # done
end_frame = [] # done
vid_dir = '/kaggle/input/wlasl2000-resized/wlasl-complete/videos'
progress = 0

for word in filtered_data:
    for i in range(5):
        vid_name = os.path.join(vid_dir, f'{word["instances"][i]["video_id"]}.mp4')
        start = word['instances'][i]['frame_start']
        end = word['instances'][i]['frame_end']
        label = word['gloss']

        video_name_list.append(vid_name)
        gloss_list.append(label)
        start_frame.append(start)
        end_frame.append(end)

        progress += 1
        print(f'Progress: {progress} / {len(filtered_data)*5}')

Progress: 1 / 50
Progress: 2 / 50
Progress: 3 / 50
Progress: 4 / 50
Progress: 5 / 50
Progress: 6 / 50
Progress: 7 / 50
Progress: 8 / 50
Progress: 9 / 50
Progress: 10 / 50
Progress: 11 / 50
Progress: 12 / 50
Progress: 13 / 50
Progress: 14 / 50
Progress: 15 / 50
Progress: 16 / 50
Progress: 17 / 50
Progress: 18 / 50
Progress: 19 / 50
Progress: 20 / 50
Progress: 21 / 50
Progress: 22 / 50
Progress: 23 / 50
Progress: 24 / 50
Progress: 25 / 50
Progress: 26 / 50
Progress: 27 / 50
Progress: 28 / 50
Progress: 29 / 50
Progress: 30 / 50
Progress: 31 / 50
Progress: 32 / 50
Progress: 33 / 50
Progress: 34 / 50
Progress: 35 / 50
Progress: 36 / 50
Progress: 37 / 50
Progress: 38 / 50
Progress: 39 / 50
Progress: 40 / 50
Progress: 41 / 50
Progress: 42 / 50
Progress: 43 / 50
Progress: 44 / 50
Progress: 45 / 50
Progress: 46 / 50
Progress: 47 / 50
Progress: 48 / 50
Progress: 49 / 50
Progress: 50 / 50


In [6]:
print(f'number of words = {len(gloss_list)}\nnumber of videos = {len(video_name_list)}')

number of words = 50
number of videos = 50


In [7]:
! pip -q install mediapipe
import mediapipe as mp
import cv2

In [8]:
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh

def detect_landmarks(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
         mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands, \
         mp_face.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        
        pose_results = pose.process(frame_rgb)
        hand_results = hands.process(frame_rgb)
        face_results = face_mesh.process(frame_rgb)

    return pose_results, hand_results, face_results

In [9]:
def extract_landmarks_from_video(video_path, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(video_path)
    if end_frame < 0:
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if start_frame < 1:
        start_frame = 1
    
    landmarks_data = []
    
    with mp_pose.Pose() as pose, mp_hands.Hands() as hands, mp_face.FaceMesh() as face_mesh:
        while cap.isOpened() and start_frame <= end_frame:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = detect_landmarks(frame)
            landmarks_data.append({
                "frame_number": start_frame,
                "body_pose_landmarks": results[0].pose_landmarks,
                "hand_landmarks": results[1].multi_hand_landmarks,
                "face_mesh_landmarks": results[2].multi_face_landmarks
            })
            start_frame += 1

    cap.release()
    return landmarks_data

In [10]:
def draw_landmarks_on_video(video_path, output_path, start_frame, end_frame, landmarks_data):
    
    cap = cv2.VideoCapture(video_path)
    if end_frame < 0:
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if start_frame < 1:
        start_frame = 1

    frame_count = 1
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened() and frame_count <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count >= start_frame:
            landmarks = landmarks_data[frame_count - start_frame]
            frame_with_landmarks = draw_landmarks(frame, landmarks)
            out.write(frame_with_landmarks)
            
        frame_count += 1

    cap.release()
    out.release()

In [11]:
def draw_landmarks(frame, landmarks):
    frame_with_landmarks = frame.copy()

    if landmarks["body_pose_landmarks"]:
        for landmark in landmarks["body_pose_landmarks"].landmark:
            x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame_with_landmarks, (x, y), 2, (0, 0, 255), -1)

    if landmarks["hand_landmarks"]:
        for hand_landmark in landmarks["hand_landmarks"]:
            for landmark in hand_landmark.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame_with_landmarks, (x, y), 2, (0, 255, 0), -1)

    if landmarks["face_mesh_landmarks"]:
        for face_landmarks in landmarks["face_mesh_landmarks"]:
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame_with_landmarks, (x, y), 2, (0, 0, 255), -1)

    return frame_with_landmarks


In [12]:
vid_dir = '/kaggle/working/new_videos'
if not os.path.exists(vid_dir):
    os.mkdir(vid_dir)

In [13]:
numpy_dir = '/kaggle/working/Numpy_files'
if not os.path.exists(numpy_dir):
    os.mkdir(numpy_dir)

In [14]:
ahh = extract_landmarks_from_video('/kaggle/input/wlasl-processed/videos/00426.mp4', 0, -1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [15]:
input_path = '/kaggle/input/wlasl-processed/videos/00426.mp4'
output_path = '/kaggle/working/ahh.mp4'

In [16]:
draw_landmarks_on_video(input_path,output_path, 0, -1, ahh)

In [17]:
type(ahh[1]['hand_landmarks']) 

NoneType

In [18]:
ahh[0]['hand_landmarks'] == None

True

In [19]:
def extract_data(frame):
    pose_landmarks = []
    face_landmarks = []
    hands_landmarks = []
    if frame['body_pose_landmarks'] != None:
        for landmark in frame['body_pose_landmarks'].landmark:
            pose_landmarks.append([landmark.x, landmark.y, landmark.z])
    else:
        pose_landmarks = np.zeros([1,33,3])

    if frame['hand_landmarks'] != None:
        for landmark in frame['hand_landmarks'][0].landmark:
            hands_landmarks.append([landmark.x, landmark.y, landmark.z])
    else:
        hands_landmarks = np.zeros([1,21,3])

    if frame['face_mesh_landmarks'] != None:
        for landmark in frame['face_mesh_landmarks'][0].landmark:
            face_landmarks.append([landmark.x, landmark.y, landmark.z])
    else:
        face_landmarks = np.zeros([1,468,3])
    if isinstance(pose_landmarks, np.ndarray):
        pose = pose_landmarks
    else:
        pose = np.array([pose_landmarks])

    if isinstance(face_landmarks, np.ndarray):
        face = face_landmarks
    else:
        face = np.array([face_landmarks])
    if isinstance(hands_landmarks, np.ndarray):
        hands = hands_landmarks
    else:
        hands = np.array([hands_landmarks])
    stacked_landmarks = np.hstack((pose, hands, face))
    return (stacked_landmarks)

In [20]:
def process_video(landmarks,start,end):
    stacked_arrays = [extract_data(landmarks[i]) for i in range(end-start)]
    temp = np.vstack(stacked_arrays)
    return temp

In [21]:
please = process_video(ahh,0,98)

In [22]:
please.shape

(98, 522, 3)

In [23]:
input_path = video_name_list[0]
output_path = '/kaggle/working/test.mp4'

In [24]:
test = extract_landmarks_from_video(input_path, start_frame[0], end_frame[0])

In [25]:
test

[{'frame_number': 887,
  'body_pose_landmarks': landmark {
    x: 0.48215633630752563
    y: 0.31225383281707764
    z: -0.5851282477378845
    visibility: 0.9999911785125732
  }
  landmark {
    x: 0.5052610039710999
    y: 0.28800511360168457
    z: -0.5424299240112305
    visibility: 0.9999668598175049
  }
  landmark {
    x: 0.518367350101471
    y: 0.28851062059402466
    z: -0.5420253276824951
    visibility: 0.9999788999557495
  }
  landmark {
    x: 0.5290775299072266
    y: 0.28906577825546265
    z: -0.5416436791419983
    visibility: 0.9999581575393677
  }
  landmark {
    x: 0.4618849754333496
    y: 0.28431999683380127
    z: -0.5357990860939026
    visibility: 0.9999730587005615
  }
  landmark {
    x: 0.44771820306777954
    y: 0.2831757664680481
    z: -0.5358351469039917
    visibility: 0.9999847412109375
  }
  landmark {
    x: 0.4350165128707886
    y: 0.2823938727378845
    z: -0.5358983874320984
    visibility: 0.9999768733978271
  }
  landmark {
    x: 0.548030674